In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
from scipy.special import jv

In [2]:
from nomaenv import NomaEnv

In [3]:
k = 4
deadlines = np.array([5 for _ in range(k)])
offsets = np.zeros(k)
arrival_probs = np.array([0.8 for _ in range(k)])
period = 5

env = NomaEnv(k,
              deadlines,
              lbda=1,
              period=period,
              arrival_probs=arrival_probs,
              offsets=offsets,
              episode_length=10,
              max_simultaneous_devices=3,
              traffic_model='aperiodic',
              channel_model='interference',
              distances=65,
              path_loss=True,
              shadowing=False,
              fast_fading=True,
              verbose=True
             )

Distances: [65. 65. 65. 65.]
Gain UE: [-90.70970328 -90.70970328 -90.70970328 -90.70970328]
n:1272.0, a:0.9999441788794395


In [22]:
done = False
obs = env.reset()
while not done:
    ac = input()
    ac = np.array([float(a) for a in ac])
    obs, reward, done = env.step(ac)

0000
attempts: [], eta: [1.12725363e-05 5.85743095e-06 3.39566502e-07 1.88662882e-06], decoding order: []
fast fading ||h||^2: [66.52608557 34.56825888  2.00398824 11.13414295]
[]
[0. 0. 0. 0.]
Timestep 1
State [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Observation [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Action [0. 0. 0. 0.]
Devices polled []
Next state [[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
Agent next obs [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Reward 0
Received packets [1. 0. 1. 1.]
Number of discarded packets 0

1011
attempts: [0 2 3], eta: [1.12622991e-05 5.93567121e-06 3.26477534e-07 1.86643622e-06], decoding order: [0 3 2]
fast fading ||h||^2: [66.46566957 35.03000221  1.92674229 11.01497412]
Device: 0, sinr 5.134644421008677, interference: [3 2]
epsilon: 0.0, decoded: 1
Device: 3, sinr 5.70849025184951, interference: [2]
epsilon: 0.0, decoded: 1
Devic

KeyboardInterrupt: Interrupted by user

array([1, 2])

In [11]:
np.random.multivariate_normal(np.array([0, 0]), 0.5 * np.diag([1, 1]), size=(4,4)).shape

(4, 4, 2)

In [19]:
mat = np.random.multivariate_normal(np.array([0, 0]), 0.5 * np.diag([1, 1]), size=(4,4))

In [24]:
mat.shape

(4, 4, 2)

In [60]:
mat[0]

array([[ 0.50993804, -0.14606585],
       [ 1.00352978, -0.45010459],
       [-1.48665428, -1.82695174],
       [-0.46292909,  0.99639939]])

In [83]:
cmat = np.matrix(mat[:,:, 0] + 1j*(mat[:, :, 1]))

In [98]:
np.sum(np.linalg.norm(cmat[0], axis=0)**2)

8.246046860407883

In [100]:
np.linalg.norm(cmat[0])**2

8.246046860407882

In [102]:
cmat[0]

matrix([[ 0.50993804-0.14606585j,  1.00352978-0.45010459j,
         -1.48665428-1.82695174j, -0.46292909+0.99639939j]])

In [133]:
np.linalg.norm(cmat.dot(cmat.getH()).diagonal(), axis=0)

array([8.24604686, 4.83584921, 6.88415179, 1.69167221])

In [137]:
cmat.dot(cmat.getH())[2,1]

(-0.8445992564455569-4.650936968394163j)

In [120]:
np.linalg.norm(cmat.dot(cmat.getH())[0,1])

1.0919039226692628

In [114]:
cmat[0].dot(cmat[0].getH())

matrix([[8.24604686+0.j]])

In [78]:
cmat[0].dot(cmat)

array([1.03732321+2.46068438j, 0.19722765+0.76798813j,
       0.79767624+0.9779176j , 1.13916179+3.31079954j])

In [30]:
np.linalg.norm(np.matrix(mat[0]).getH().dot(mat[1]))

1.4851618309773036

In [31]:
np.matrix(mat[0]).shape

(4, 2)

In [42]:
np.linalg.norm(np.matrix(mat[0]).getH().dot(np.matrix(mat[0])), axis=1)

array([4.07622879, 4.87149774])

In [53]:
np.linalg.norm(mat[0], axis=1).dot(np.linalg.norm(mat, axis=-1).T)

array([8.24604686, 4.29300546, 5.21002229, 2.73788792])

In [49]:
(np.linalg.norm(mat[0], axis=1)*normmat[1]).sum()

4.293005458210736

In [54]:
normmat = np.linalg.norm(mat, axis=-1)

In [58]:
(np.linalg.norm(mat[0], axis=1)**2).sum()

8.246046860407883

In [59]:
normmat.dot(normmat.T)

array([[8.24604686, 4.29300546, 5.21002229, 2.73788792],
       [4.29300546, 4.83584921, 5.65600271, 2.6822529 ],
       [5.21002229, 5.65600271, 6.88415179, 3.1215787 ],
       [2.73788792, 2.6822529 , 3.1215787 , 1.69167221]])

In [138]:
np.delete(np.array([1,2,3,4]), [2,3])

array([1, 2])